In [5]:
#Python 3 enviroment for blockchain program using Flask to run the actual server and postman for http requests
#standard imports

import pandas as pd
import numpy as np

#imports necessary for talking with and manipulating the blockchain
from bitcoin import *
import hashlib
import json
from textwrap import dedent
from time import time
from uuid import uuid4
from flask import Flask, jsonify, request
from blockchain import exchangerates as er

In [2]:
#as of November 1st what is bitcoin trading at?
from blockchain import exchangerates as er
rates = er.get_ticker()
for i in rates:
    print(i, rates[i].p15min)

USD 6709.54
AUD 8740.77
BRL 21923.41
CAD 8630.74
CHF 6727.0
CLP 4261225.68
CNY 44746.99
DKK 42949.5
EUR 5802.58
GBP 5062.02
HKD 52338.73
INR 433190.39
ISK 713357.76
JPY 767914.8
KRW 7454897.24
NZD 9736.94
PLN 24429.27
RUB 390813.64
SEK 56361.2
SGD 9130.35
THB 222353.99
TWD 202245.51


In [3]:
from blockchain import statistics

stats = statistics.get()
print(stats.trade_volume_btc)
print(stats.total_btc)
print(stats.market_price_usd)

94521.86
1665881250000000
6665.3


In [4]:
#setup the chain class
class Blockchain(object):
    def __init__(self):
        self.chain = []
        self.current_transactions = []
        #create genes block
        self.new_block(previous_hash=1, proof =100)
        
    def new_block(self, proof, previous_hash=None):
        """
        Create a new Block in the chain
        :param proof: <int> The proof given by the Proof of Work algorithm
        :param previous_hash: (Optional) <str> Hash of previous Block
        :return: <dict> New Block
        """
        block = {
            'index': len(self.chain)+1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }
        #reset current list of transactions
        self.current_transactions = []
        self.chain.append(block)
        return block
    
    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: <str> Address of the Sender
        :param recipient: <str> Address of the Recipient
        :param amount: <int> Amount
        :return: <int> The index of the Block that will hold this transaction
        """

        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })

        return self.last_block['index'] + 1
    def proof_of_work(self, last_proof):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes, where p is the previous p'
         - p is the previous proof, and p' is the new proof
        :param last_proof: <int>
        :return: <int>
        """

        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1

        return proof
    
    @staticmethod
    def hash(block):
        """
        Creates a sha256 hash of a Block
        :param block: <dict> Block
        :return: <str>        
        """
        #check for ordered dictionairy for consistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    def valid_proof(last_proof, proof):
        """
        Validates the Proof: Does hash(last_proof, proof) contain 4 leading zeroes?
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :return: <bool> True if correct, False if not.
        """

        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

    @property
    def last_block(self):
        # Returns the last Block in the chain
        return self.chain[-1]

In [ ]:
# Instantiate our Node
app = Flask(__name__)

# Generate a globally unique address for this node
node_identifier = str(uuid4()).replace('-', '')

# Instantiate the Blockchain
blockchain = Blockchain()



@app.route('/mine', methods=['GET'])
def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    block = blockchain.new_block(proof)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
return jsonify(response), 200
  
@app.route('/transactions/new', methods=['POST'])
def new_transaction():
    values = request.get_json()

    # Check that the required fields are in the POST'ed data
    required = ['sender', 'recipient', 'amount']
    if not all(k in values for k in required):
        return 'Missing values', 400

    # Create a new Transaction
    index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])

    response = {'message': f'Transaction will be added to Block {index}'}
return jsonify(response), 201

@app.route('/chain', methods=['GET'])
def full_chain():
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [ ]:
#written from tutorial posted on https://hackernoon.com/learn-blockchains-by-building-one-117428612f46